# Total dynamical O-information

## Index

1. [Functions](#functions)
1. [Load and binarize data](#load_binar_data)
1. [Full length](#full_length)
    1. [Thomas' work](#thomas_work_full)
    1. [3 vs 1-2](#3_vs_1-2_full)
        1. [J-S divergence](#J-S_divergence_full)
    1. [3 vs 1-1-1](#3_vs_1-1-1_full)
1. [Half length](#half_length)
    1. [Thomas' work](#thomas_work_half)
    1. [3 vs 1-2](#3_vs_1-2_half)
        1. [J-S divergence](#J-S_divergence_half)
    1. [3 vs 1-1-1](#3_vs_1-1-1_half)
1. [Comparison different lengths](#comparison_lengths)
    1. [Comparison full-half](#comparison_f_h)
    1. [Creation dataset](#creation_dataset)
        1. [Comparison Industrials](#comparison_industrials)
        1. [Comparison Basic Materials](#comparison_basic_materials)
        1. [Comparison Financials](#comparison_financials)
        1. [Comparison Consumer Services](#comparison_consumer_services)
        1. [Comparison Health Care](#comparison_health_care)
        1. [Comparison Utilities](#comparison_utilities)
        1. [Comparison Oil & Gas](#comparison_oil_gas)
        1. [Comparison Consumer Goods](#comparison_consumer_goods)

<a name="functions"></a>
## Functions

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import combinations
from tqdm.auto import tqdm
import matplotlib.patches as mpatches
import pickle
from scipy.special import kl_div
from scipy.stats import entropy

from ho_info_metrics.metrics import *
from ho_info_metrics.utils import *

In [2]:
def binarize_df(df, window_size):
    df = df.drop(columns=["Date"])
    for col in df.columns:
        if col != "Date":
            col_values = df[col].values
            binarized_values = []
            for i in range(len(col_values)):
                window_start = max(0, i - window_size + 1)
                window_mean = col_values[window_start : i + 1].mean()
                binarized_values.append(1 if col_values[i] > window_mean else 0)
            df[col] = binarized_values
    return df

In [3]:
def get_do_infos(target_sector, binarized_financial_ts):

    print(">>>", target_sector)
    target_stocks = list(
        stock_names[stock_names["Sector"] == target_sector]["Stock name"]
    )  # tutti gli stock name il cui settore sia il target
    print(len(target_stocks))
    other_stocks = list(
        stock_names[stock_names["Sector"] != target_sector]["Stock name"]
    )
    print(len(other_stocks))

    same_sector_triplets = list(combinations(target_stocks, 3))
    print(len(same_sector_triplets))
    different_sector_triplets = []
    while len(different_sector_triplets) != len(same_sector_triplets):
        triplet = []
        triplet.append(np.random.choice(target_stocks))
        triplet.append(np.random.choice(other_stocks))
        triplet.append(np.random.choice(other_stocks))
        different_sector_triplets.append(triplet)
    print(len(different_sector_triplets))

    doinfo_same_sector = []
    doinfo_different_sector = []
    bar_length = len(same_sector_triplets)
    with tqdm(total=bar_length) as pbar:
        pbar.set_description("Same sector triplets")
        for i in range(len(same_sector_triplets)):
            X1 = binarized_financial_ts[same_sector_triplets[i][0]].values
            X2 = binarized_financial_ts[same_sector_triplets[i][1]].values
            X3 = binarized_financial_ts[same_sector_triplets[i][2]].values
            X = np.vstack((X1, X2, X3))
            doinfo_same_sector.append(
                o_information_lagged_all(X, estimator="cat_ent")
            )  # calcola la total dynamical o_information della tripletta
            pbar.update(1)

    bar_length = len(different_sector_triplets)
    with tqdm(total=bar_length) as pbar:
        for i in range(len(different_sector_triplets)):
            X1 = binarized_financial_ts[different_sector_triplets[i][0]].values
            X2 = binarized_financial_ts[different_sector_triplets[i][1]].values
            X3 = binarized_financial_ts[different_sector_triplets[i][2]].values
            X = np.vstack((X1, X2, X3))
            doinfo_different_sector.append(
                o_information_lagged_all(X, estimator="cat_ent")
            )
            pbar.update(1)

    return (
        doinfo_same_sector,
        doinfo_different_sector,
    )  # ogni termine ritorna un array con la total dynamical o_information per ogni tripletta

In [4]:
def get_do_infos_same(target_sector, binarized_financial_ts):
    """Get the total dynamical o_information for the triplets of the target sector"""

    print(">>>", target_sector)

    target_stocks = list(
        stock_names[stock_names["Sector"] == target_sector]["Stock name"]
    )  # tutti gli stock name il cui settore sia il target
    print("Length target_stocks:", len(target_stocks))

    # Having removed the Technology sector, each sector can now offer 35 triplets
    same_sector_triplets = list(combinations(target_stocks, 3))[:35]
    print("Length triplets same sector:", len(same_sector_triplets))

    doinfo_one_sector = []
    bar_length = len(same_sector_triplets)
    with tqdm(total=bar_length) as pbar:
        pbar.set_description("Same sector")
        for i in range(len(same_sector_triplets)):
            X1 = binarized_financial_ts[same_sector_triplets[i][0]].values
            X2 = binarized_financial_ts[same_sector_triplets[i][1]].values
            X3 = binarized_financial_ts[same_sector_triplets[i][2]].values
            X = np.vstack((X1, X2, X3))
            doinfo_one_sector.append(
                o_information_lagged_all(X, estimator="cat_ent")
            )  # calcola la total dynamical o_information della tripletta
            pbar.update(1)

    return doinfo_one_sector

In [5]:
def get_do_infos_two_different(target_sector, another_sector, binarized_financial_ts):
    """Get the total dynamical o_information for the triplets
    composed of one node in the target_sector and two in the another_sector"""

    print(">>>", target_sector, "-", another_sector)

    target_stocks = list(
        stock_names[stock_names["Sector"] == target_sector]["Stock name"]
    )
    another_stocks = list(
        stock_names[stock_names["Sector"] == another_sector]["Stock name"]
    )
    print("Length another_stocks:", len(another_stocks))

    # Having removed the Technology sector, each sector can now offer 35 triplets
    another_sector_triplets = []
    while len(another_sector_triplets) != 35:
        triplet = []
        triplet.append(np.random.choice(target_stocks))
        triplet.append(np.random.choice(another_stocks))
        triplet.append(np.random.choice(another_stocks))
        another_sector_triplets.append(triplet)
    print("Length triplets two sectors:", len(another_sector_triplets))

    doinfo_two_sectors = []
    bar_length = len(another_sector_triplets)
    with tqdm(total=bar_length) as pbar:
        pbar.set_description("Two sectors")
        for i in range(len(another_sector_triplets)):
            X1 = binarized_financial_ts[another_sector_triplets[i][0]].values
            X2 = binarized_financial_ts[another_sector_triplets[i][1]].values
            X3 = binarized_financial_ts[another_sector_triplets[i][2]].values
            X = np.vstack((X1, X2, X3))
            doinfo_two_sectors.append(o_information_lagged_all(X, estimator="cat_ent"))
            pbar.update(1)

    return doinfo_two_sectors

In [6]:
def get_do_infos_three_different(
    target_sector, sector1, sector2, binarized_financial_ts
):
    """Get the total dynamical o_information for the triplets
    composed of three nodes in three different sectors"""

    print(">>>", target_sector, "-", sector1, "-", sector2)

    target_stocks = list(
        stock_names[stock_names["Sector"] == target_sector]["Stock name"]
    )
    stocks1 = list(stock_names[stock_names["Sector"] == sector1]["Stock name"])
    print("Length sector1:", len(stocks1))
    stocks2 = list(stock_names[stock_names["Sector"] == sector2]["Stock name"])
    print("Length sector2:", len(stocks2))

    # Having removed the Technology sector, each sector can now offer 35 triplets
    three_sectors_triplets = []
    while len(three_sectors_triplets) != 35:
        triplet = []
        triplet.append(np.random.choice(target_stocks))
        triplet.append(np.random.choice(stocks1))
        triplet.append(np.random.choice(stocks2))
        three_sectors_triplets.append(triplet)
    print("Length triplets three sectors:", len(three_sectors_triplets))

    doinfo_three_sectors = []
    bar_length = len(three_sectors_triplets)
    with tqdm(total=bar_length) as pbar:
        pbar.set_description("Three sectors")
        for i in range(len(three_sectors_triplets)):
            X1 = binarized_financial_ts[three_sectors_triplets[i][0]].values
            X2 = binarized_financial_ts[three_sectors_triplets[i][1]].values
            X3 = binarized_financial_ts[three_sectors_triplets[i][2]].values
            X = np.vstack((X1, X2, X3))
            doinfo_three_sectors.append(
                o_information_lagged_all(X, estimator="cat_ent")
            )
            pbar.update(1)

    return doinfo_three_sectors

In [7]:
def max_bins(array1, array2, array3):
    """Return the maximum number of bins so that there is at least one value inside each interval"""
    
    bins = 1
    cond = True

    while cond:
        bins += 1
        array1_hist, _ = np.histogram(array1, bins=bins)
        array2_hist, _ = np.histogram(array2, bins=bins)
        array3_hist, _ = np.histogram(array3, bins=bins)

        if np.isin(0, array1_hist):
            cond = False

        elif np.isin(0, array2_hist):
            cond = False

        elif np.isin(0, array3_hist):
            cond = False
            
    return bins - 1

In [8]:
def JS_div(array1, array2):
    """Compute the J-S divergence of two array"""
    
    array1_sort = np.sort(array1)
    array2_sort = np.sort(array2)

    array1_normalized = array1_sort / np.sum(array1_sort)
    array2_normalized = array2_sort / np.sum(array2_sort)

    average = (array1_normalized + array2_normalized) / 2
    average_sort = np.sort(average)
    
    bins = max_bins(array1_normalized, array2_normalized, average_sort)
    
    array1_hist, _ = np.histogram(array1_normalized, bins=bins)
    array2_hist, _ = np.histogram(array2_normalized, bins=bins)
    average_hist, _ = np.histogram(average_sort, bins=bins)
    
    kl_array1 = entropy(array1_hist, average_hist)
    kl_array2 = entropy(array2_hist, average_hist)
 
    return np.sum(kl_array1 + kl_array2) / 2

<a name="load_binar_data"></a>
## Load and binarize data

In [9]:
financial_ts = pd.read_csv(
    "data/Economic_data/NYSE_119stocks_2000Jan_2021June_withdates.csv"
)
stock_names = pd.read_csv(
    "data/Economic_data/list_stocknames.txt",
    sep=", ",
    names=["Index", "Stock name", "Sector"],
)

C:\Users\david\AppData\Local\Temp\ipykernel_10764\1982450507.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  stock_names = pd.read_csv(


In [10]:
financial_ts

,Date,A,AA,AB,ABC,ABT,ADC,AEO,AEP,AES,...,DHI,DIN,DIS,DLX,DNP,DOV,DRE,DRI,DRQ,DSM
0,1999-12-31 00:00:00,55.302216,99.724503,29.937500,3.796875,16.300713,14.250000,10.000000,32.125000,37.375000,...,3.805411,16.687500,28.855125,21.936281,8.3125,30.416479,19.500000,10.801292,15.187500,7.3750
1,2000-01-03 00:00:00,51.502148,97.246407,29.750000,3.890625,15.711531,13.937500,9.777778,31.437500,36.250000,...,3.650439,16.125000,29.471687,21.436594,8.3125,29.410976,19.000000,10.317096,15.125000,7.4375
2,2000-01-04 00:00:00,47.567955,97.696968,29.625000,3.625000,15.262630,13.562500,8.916667,31.812500,34.812500,...,3.512687,16.062500,31.198063,21.036844,8.5000,28.866327,18.875000,10.242604,15.000000,7.5000
3,2000-01-05 00:00:00,44.617310,103.329002,29.750000,3.906250,15.234574,13.312500,9.590278,33.000000,35.187500,...,3.564344,16.250000,32.492844,20.836969,8.8750,28.992016,18.937500,10.615063,14.843750,7.5625
4,2000-01-06 00:00:00,42.918453,101.977310,30.250000,4.203125,15.767643,13.562500,8.909722,33.187500,35.468750,...,3.374935,17.562500,31.198063,20.787001,9.0000,29.117704,19.375000,10.503325,14.500000,7.5625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5395,2021-06-11 00:00:00,143.529999,37.380001,45.900002,119.839996,109.910004,71.559998,34.320000,84.760002,25.879999,...,88.730003,92.830002,177.380005,47.619999,10.5400,151.759995,49.299999,139.110001,37.750000,8.1400
5396,2021-06-14 00:00:00,144.389999,36.950001,45.509998,118.730003,110.480003,72.169998,34.759998,85.129997,25.740000,...,87.980003,91.500000,178.179993,46.820000,10.6100,150.339996,49.500000,137.500000,37.880001,8.1600
5397,2021-06-15 00:00:00,144.619995,36.290001,45.349998,119.029999,110.410004,70.489998,34.209999,85.239998,26.059999,...,88.660004,90.940002,175.860001,46.779999,10.6100,151.369995,48.900002,136.710007,39.090000,8.1900
5398,2021-06-16 00:00:00,143.589996,35.360001,45.340000,118.360001,110.059998,70.320000,34.740002,84.400002,25.730000,...,87.330002,91.570000,174.660004,46.180000,10.5500,149.990005,48.119999,136.460007,39.720001,8.1500


In [11]:
stock_names

,Index,Stock name,Sector
0,0,A,Industrials
1,1,AA,Basic Materials
2,2,AB,Financials
3,3,ABC,Consumer Services
4,4,ABT,Health Care
...,...,...,...
114,114,DOV,Industrials
115,115,DRE,Financials
116,116,DRI,Consumer Services
117,117,DRQ,Oil & Gas


In [12]:
tw = 7  # time window, originalmente era 2
binarized_financial_ts = binarize_df(financial_ts, tw)

In [13]:
binarized_financial_ts

,A,AA,AB,ABC,ABT,ADC,AEO,AEP,AES,AFG,...,DHI,DIN,DIS,DLX,DNP,DOV,DRE,DRI,DRQ,DSM
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,1
3,0,1,0,1,0,0,1,1,0,0,...,0,0,1,0,1,0,0,1,0,1
4,0,1,1,1,1,0,0,1,0,0,...,0,1,1,0,1,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5395,1,0,1,1,1,0,1,0,1,0,...,0,0,1,1,1,1,1,1,1,0
5396,1,0,0,1,1,0,1,1,1,0,...,0,0,1,0,1,0,1,0,1,0
5397,1,0,0,1,1,0,1,1,1,1,...,0,0,0,0,1,1,0,0,1,1
5398,1,0,0,0,1,0,1,0,1,1,...,0,0,0,0,1,0,0,0,1,0


In [14]:
sectors = list(stock_names["Sector"].unique())
sectors

['Industrials',
 'Basic Materials',
 'Financials',
 'Consumer Services',
 'Health Care',
 'Utilities',
 'Oil & Gas',
 'Consumer Goods',
 'Technology']

<a name="full_length"></a>
## Full length

We consider the full length of the time series.

<a name="thomas_work_full"></a>
### Thomas' work

Each sector contains the maximun number of triplets, so the number of triplets is not constant.

In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE DATA

for i in range(len(sectors)):
    doinfo_same_sector, doinfo_different_sector = get_do_infos(
        sectors[i], binarized_financial_ts
    )

    np.save(
        f"./results/economic_results/full/do_info/doinfo_same_sector_{sectors[i]}_tw_{tw}.npy",
        doinfo_same_sector,
    )

    np.save(
        f"./results/economic_results/full/do_info/doinfo_different_sector_{sectors[i]}_tw_{tw}.npy",
        doinfo_different_sector,
    )

In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE VIOLIN PLOT

fig, axs = plt.subplots(3, 3, figsize=(40, 40))
for i in range(len(sectors)):
    row = i // 3
    col = i % 3
    doinfo_same_sector = np.load(
        f"./results/economic_results/full/do_info/doinfo_same_sector_{sectors[i]}_tw_{tw}.npy"
    )
    doinfo_different_sector = np.load(
        f"./results/economic_results/full/do_info/doinfo_different_sector_{sectors[i]}_tw_{tw}.npy"
    )
    parts = axs[row, col].violinplot(doinfo_same_sector, showextrema=False)
    for pc in parts["bodies"]:
        pc.set_facecolor("tab:blue")
        pc.set_edgecolor("black")
        pc.set_alpha(0.5)
    parts = axs[row, col].violinplot(doinfo_different_sector, showextrema=False)
    for pc in parts["bodies"]:
        pc.set_facecolor("tab:orange")
        pc.set_edgecolor("black")
        pc.set_alpha(0.5)
    axs[row, col].set_title(sectors[i], size=35)
    axs[row, col].set_ylabel(r"$d\Omega_3^{tot.}$", size=37)
    axs[row, col].tick_params(axis="both", labelsize=18)
    axs[row, col].legend(
        handles=[
            mpatches.Patch(color="tab:blue", label="Same sector", alpha=0.4),
            mpatches.Patch(color="tab:orange", label="Different sector", alpha=0.4),
        ],
        fontsize=30,
    )


plt.savefig(
    f"./results/economic_results/full/images/doinfo_random_sectors_tw_{tw}.pdf",
    dpi=600,
    bbox_inches="tight",
)

<a name="3_vs_1-2_full"></a>
### 3 vs 1-2

In previous work, a triplet from the same sector was compared with a triplet in which one node is from the target sector and the other two are randomly selected from other sectors. Now I would like to compare the triplet from the same sector with a more carefully chosen triplet (1 from the target sector and the other two from tho different chosen sectors).
  
To ensure that the number of triplets is always the same I take 35, which is the maximum number that all sectors can offer.

In [15]:
target_sector = "Technology"
list(stock_names[stock_names["Sector"] == target_sector]["Stock name"])

['CACI', 'CIEN', 'DBD', 'DDD']

In [16]:
# I am deleting the data for the Techonolgy sector because there are only a few
binarized_financial_ts = binarized_financial_ts.drop(
    columns=["CACI", "CIEN", "DBD", "DDD"]
)

In [17]:
# Usa il metodo .isin() per creare una maschera booleana
maschera = stock_names["Stock name"].isin(["CACI", "CIEN", "DBD", "DDD"])

# Usa la maschera per selezionare solo le righe che non soddisfano la condizione
stock_names = stock_names[~maschera]

In [18]:
sectors = sectors[:-1]
sectors

['Industrials',
 'Basic Materials',
 'Financials',
 'Consumer Services',
 'Health Care',
 'Utilities',
 'Oil & Gas',
 'Consumer Goods']

In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE DATA

for target in sectors:
    other_sectors = [sector for sector in sectors if sector != target]
    for i, sector in enumerate(other_sectors):

        if i == 0:
            doinfo_one_sector = get_do_infos_same(target, binarized_financial_ts)
            np.save(
                f"./results/economic_results/full/do_info/doinfo_one_sector_{target}_tw_{tw}.npy",
                doinfo_one_sector,
            )

        doinfo_two_sectors = get_do_infos_two_different(
            target, sector, binarized_financial_ts
        )
        np.save(
            f"./results/economic_results/full/do_info/doinfo_two_sectors_{target}_{sector}_tw_{tw}.npy",
            doinfo_two_sectors,
        )

In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE VIOLIN PLOTS

for target in sectors:
    other_sectors = [sector for sector in sectors if sector != target]
    fig, axs = plt.subplots(3, 3, figsize=(40, 40))
    for i, sector in enumerate(other_sectors):
        row = i // 3
        col = i % 3
        doinfo_one_sector = np.load(
            f"./results/economic_results/full/do_info/doinfo_one_sector_{target}_tw_{tw}.npy"
        )
        doinfo_two_sectors = np.load(
            f"./results/economic_results/full/do_info/doinfo_two_sectors_{target}_{sector}_tw_{tw}.npy"
        )
        parts = axs[row, col].violinplot(doinfo_one_sector, showextrema=False)
        for pc in parts["bodies"]:
            pc.set_facecolor("tab:blue")
            pc.set_edgecolor("black")
            pc.set_alpha(0.5)
        parts = axs[row, col].violinplot(doinfo_two_sectors, showextrema=False)
        for pc in parts["bodies"]:
            pc.set_facecolor("tab:orange")
            pc.set_edgecolor("black")
            pc.set_alpha(0.5)
        title = str(target + " - " + sector)
        axs[row, col].set_title(title, size=35)
        axs[row, col].set_ylabel(r"$d\Omega_3^{tot.}$", size=40)
        axs[row, col].tick_params(axis="both", labelsize=18)
        label1 = str("3 in " + target)
        label2 = str("1 in " + target + ", 2 in " + sector)
        axs[row, col].legend(
            handles=[
                mpatches.Patch(color="tab:blue", label=label1, alpha=0.4),
                mpatches.Patch(color="tab:orange", label=label2, alpha=0.4),
            ],
            fontsize=25,
        )

    plt.savefig(
        f"./results/economic_results/full/images/two_sectors/doinfo_two_sectors_{target}_tw_{tw}.pdf",
        dpi=600,
        bbox_inches="tight",
    )

<a name="J-S_divergence_full"></a>
#### J-S divergence

Let's summarize the difference between the two distributions into a single number, so we can make a comparison.

In [ ]:
# The row index indicates the target sector, the column index the other sector
df_full = pd.DataFrame(
    columns=[
        "Industrials",
        "Basic Materials",
        "Financials",
        "Consumer Services",
        "Health Care",
        "Utilities",
        "Oil & Gas",
        "Consumer Goods",
    ]
)

In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE DATAFRAME

for target in sectors:
    other_sectors = [sector for sector in sectors if sector != target]

    for i, sector in enumerate(other_sectors):
        doinfo_one_sector = np.load(
            f"./results/economic_results/full/do_info/doinfo_one_sector_{target}_tw_{tw}.npy"
        )
        doinfo_two_sectors = np.load(
            f"./results/economic_results/full/do_info/doinfo_two_sectors_{target}_{sector}_tw_{tw}.npy"
        )

        df_full.at[target, sector] = JS_div(doinfo_one_sector, doinfo_two_sectors)

In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE DATAFRAME

df_full.to_csv("./results/economic_results/full/result_full.csv", index=True)
df_full

<a name="3_vs_1-1-1_full"></a>
### 3 vs 1-1-1

The triplet from the same sector is compared with a triplet in which each element came from a different sector (one must be the target sector).
  
To ensure that the number of triplets is always the same I take 35, which is the maximum number that all sectors can offer.

In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE DATA

for target in sectors:
    other_sectors = [sector for sector in sectors if sector != target]

    for i, sector1 in enumerate(other_sectors):
        other_sectors2 = [sector for sector in other_sectors if sector != sector1]
        for j, sector2 in enumerate(other_sectors2):

            doinfo_three_sectors = get_do_infos_three_different(
                target, sector1, sector2, binarized_financial_ts
            )
            np.save(
                f"./results/economic_results/full/do_info/doinfo_three_sectors_{target}_{sector1}_{sector2}_tw_{tw}.npy",
                doinfo_three_sectors,
            )

In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE VIOLIN PLOTS

for target1 in sectors:
    for target2 in [sector for sector in sectors if sector != target1]:
        other_sectors = [
            sector for sector in sectors if sector != target1 and sector != target2
        ]

        fig, axs = plt.subplots(3, 2, figsize=(40, 40))
        for i, sector in enumerate(other_sectors):
            row = i // 2
            col = i % 2
            doinfo_one_sector = np.load(
                f"./results/economic_results/full/do_info/doinfo_one_sector_{target1}_tw_{tw}.npy"
            )
            doinfo_three_sectors = np.load(
                f"./results/economic_results/full/do_info/doinfo_three_sectors_{target1}_{target2}_{sector}_tw_{tw}.npy"
            )
            parts = axs[row, col].violinplot(doinfo_one_sector, showextrema=False)
            for pc in parts["bodies"]:
                pc.set_facecolor("tab:blue")
                pc.set_edgecolor("black")
                pc.set_alpha(0.5)
            parts = axs[row, col].violinplot(doinfo_three_sectors, showextrema=False)
            for pc in parts["bodies"]:
                pc.set_facecolor("tab:orange")
                pc.set_edgecolor("black")
                pc.set_alpha(0.5)
            title = str(target1 + " - " + target2 + " - " + sector)
            axs[row, col].set_title(title, size=40)
            axs[row, col].set_ylabel(r"$d\Omega_3^{tot.}$", size=45)
            axs[row, col].tick_params(axis="both", labelsize=25)
            label1 = str("3 in " + target1)
            label2 = str("1 in " + target1 + ", 1 in " + target2 + ", 1 in " + sector)
            axs[row, col].legend(
                handles=[
                    mpatches.Patch(color="tab:blue", label=label1, alpha=0.5),
                    mpatches.Patch(color="tab:orange", label=label2, alpha=0.5),
                ],
                fontsize=25,
            )

        plt.savefig(
            f"./results/economic_results/full/images/three_sectors/doinfo_three_sectors_{target1}_{target2}_tw_{tw}.pdf",
            dpi=600,
            bbox_inches="tight",
        )

<a name="half_length"></a>
## Half length

I only take half time series, to see if the differences between the distributions decrease.

In [ ]:
half_length = int(len(binarized_financial_ts) / 2)
binarized_financial_ts_half = binarized_financial_ts[:half_length]
binarized_financial_ts_half

<a name="thomas_work_half"></a>
### Thomas' work

In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE DATA

for i in range(len(sectors)):
    doinfo_same_sector, doinfo_different_sector = get_do_infos(
        sectors[i], binarized_financial_ts_half
    )

    np.save(
        f"./results/economic_results/half/do_info/doinfo_same_sector_{sectors[i]}_tw_{tw}.npy",
        doinfo_same_sector,
    )

    np.save(
        f"./results/economic_results/half/do_info/doinfo_different_sector_{sectors[i]}_tw_{tw}.npy",
        doinfo_different_sector,
    )

In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE VIOLIN PLOT

fig, axs = plt.subplots(3, 3, figsize=(40, 40))
for i in range(len(sectors)):
    row = i // 3
    col = i % 3
    doinfo_same_sector = np.load(
        f"./results/economic_results/half/do_info/doinfo_same_sector_{sectors[i]}_tw_{tw}.npy"
    )
    doinfo_different_sector = np.load(
        f"./results/economic_results/half/do_info/doinfo_different_sector_{sectors[i]}_tw_{tw}.npy"
    )
    parts = axs[row, col].violinplot(doinfo_same_sector, showextrema=False)
    for pc in parts["bodies"]:
        pc.set_facecolor("tab:blue")
        pc.set_edgecolor("black")
        pc.set_alpha(0.5)
    parts = axs[row, col].violinplot(doinfo_different_sector, showextrema=False)
    for pc in parts["bodies"]:
        pc.set_facecolor("tab:orange")
        pc.set_edgecolor("black")
        pc.set_alpha(0.5)
    axs[row, col].set_title(sectors[i], size=35)
    axs[row, col].set_ylabel(r"$d\Omega_3^{tot.}$", size=37)
    axs[row, col].tick_params(axis="both", labelsize=18)
    axs[row, col].legend(
        handles=[
            mpatches.Patch(color="tab:blue", label="Same sector", alpha=0.4),
            mpatches.Patch(color="tab:orange", label="Different sector", alpha=0.4),
        ],
        fontsize=30,
    )


plt.savefig(
    f"./results/economic_results/half/images/doinfo_random_sectors_tw_{tw}.pdf",
    dpi=600,
    bbox_inches="tight",
)

<a name="3_vs_1-2_half"></a>
### 3 vs 1-2

!!!! Run the initial part of the full 3 vs 1-2 first, so as to eliminate the stocks relating to technology

In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE DATA

for target in sectors:
    other_sectors = [sector for sector in sectors if sector != target]
    for i, sector in enumerate(other_sectors):

        if i == 0:
            doinfo_one_sector = get_do_infos_same(target, binarized_financial_ts_half)
            np.save(
                f"./results/economic_results/half/do_info/doinfo_one_sector_{target}_tw_{tw}.npy",
                doinfo_one_sector,
            )

        doinfo_two_sectors = get_do_infos_two_different(
            target, sector, binarized_financial_ts_half
        )
        np.save(
            f"./results/economic_results/half/do_info/doinfo_two_sectors_{target}_{sector}_tw_{tw}.npy",
            doinfo_two_sectors,
        )

In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE VIOLIN PLOTS

for target in sectors:
    other_sectors = [sector for sector in sectors if sector != target]
    fig, axs = plt.subplots(3, 3, figsize=(40, 40))
    for i, sector in enumerate(other_sectors):
        row = i // 3
        col = i % 3
        doinfo_one_sector = np.load(
            f"./results/economic_results/half/do_info/doinfo_one_sector_{target}_tw_{tw}.npy"
        )
        doinfo_two_sectors = np.load(
            f"./results/economic_results/half/do_info/doinfo_two_sectors_{target}_{sector}_tw_{tw}.npy"
        )
        parts = axs[row, col].violinplot(doinfo_one_sector, showextrema=False)
        for pc in parts["bodies"]:
            pc.set_facecolor("tab:blue")
            pc.set_edgecolor("black")
            pc.set_alpha(0.5)
        parts = axs[row, col].violinplot(doinfo_two_sectors, showextrema=False)
        for pc in parts["bodies"]:
            pc.set_facecolor("tab:orange")
            pc.set_edgecolor("black")
            pc.set_alpha(0.5)
        title = str(target + " - " + sector)
        axs[row, col].set_title(title, size=35)
        axs[row, col].set_ylabel(r"$d\Omega_3^{tot.}$", size=40)
        axs[row, col].tick_params(axis="both", labelsize=18)
        label1 = str("3 in " + target)
        label2 = str("1 in " + target + ", 2 in " + sector)
        axs[row, col].legend(
            handles=[
                mpatches.Patch(color="tab:blue", label=label1, alpha=0.4),
                mpatches.Patch(color="tab:orange", label=label2, alpha=0.4),
            ],
            fontsize=25,
        )

    plt.savefig(
        f"./results/economic_results/half/images/two_sectors/doinfo_two_sectors_{target}_tw_{tw}.pdf",
        dpi=600,
        bbox_inches="tight",
    )

<a name="J-S_divergence_half"></a>
#### J-S divergence

In [ ]:
# The row index indicates the target sector, the column index the other sector
df_half = pd.DataFrame(
    columns=[
        "Industrials",
        "Basic Materials",
        "Financials",
        "Consumer Services",
        "Health Care",
        "Utilities",
        "Oil & Gas",
        "Consumer Goods",
    ]
)

In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE DATAFRAME

for target in sectors:
    other_sectors = [sector for sector in sectors if sector != target]

    for i, sector in enumerate(other_sectors):
        doinfo_one_sector = np.load(
            f"./results/economic_results/half/do_info/doinfo_one_sector_{target}_tw_{tw}.npy"
        )
        doinfo_two_sectors = np.load(
            f"./results/economic_results/half/do_info/doinfo_two_sectors_{target}_{sector}_tw_{tw}.npy"
        )

        df_half.at[target, sector] = JS_div(doinfo_one_sector, doinfo_two_sectors)

In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE DATAFRAME

df_half.to_csv("./results/economic_results/half/result_full.csv", index=True)
df_half

<a name="3_vs_1-1-1_half"></a>
### 3 vs 1-1-1

In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE DATA

for target in sectors:
    other_sectors = [sector for sector in sectors if sector != target]

    for i, sector1 in enumerate(other_sectors):
        other_sectors2 = [sector for sector in other_sectors if sector != sector1]
        for j, sector2 in enumerate(other_sectors2):

            doinfo_three_sectors = get_do_infos_three_different(
                target, sector1, sector2, binarized_financial_ts_half
            )
            np.save(
                f"./results/economic_results/half/do_info/doinfo_three_sectors_{target}_{sector1}_{sector2}_tw_{tw}.npy",
                doinfo_three_sectors,
            )

In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE VIOLIN PLOTS

for target1 in sectors:
    for target2 in [sector for sector in sectors if sector != target1]:
        other_sectors = [
            sector for sector in sectors if sector != target1 and sector != target2
        ]

        fig, axs = plt.subplots(3, 2, figsize=(40, 40))
        for i, sector in enumerate(other_sectors):
            row = i // 2
            col = i % 2
            doinfo_one_sector = np.load(
                f"./results/economic_results/half/do_info/doinfo_one_sector_{target1}_tw_{tw}.npy"
            )
            doinfo_three_sectors = np.load(
                f"./results/economic_results/half/do_info/doinfo_three_sectors_{target1}_{target2}_{sector}_tw_{tw}.npy"
            )
            parts = axs[row, col].violinplot(doinfo_one_sector, showextrema=False)
            for pc in parts["bodies"]:
                pc.set_facecolor("tab:blue")
                pc.set_edgecolor("black")
                pc.set_alpha(0.5)
            parts = axs[row, col].violinplot(doinfo_three_sectors, showextrema=False)
            for pc in parts["bodies"]:
                pc.set_facecolor("tab:orange")
                pc.set_edgecolor("black")
                pc.set_alpha(0.5)
            title = str(target1 + " - " + target2 + " - " + sector)
            axs[row, col].set_title(title, size=40)
            axs[row, col].set_ylabel(r"$d\Omega_3^{tot.}$", size=45)
            axs[row, col].tick_params(axis="both", labelsize=25)
            label1 = str("3 in " + target1)
            label2 = str("1 in " + target1 + ", 1 in " + target2 + ", 1 in " + sector)
            axs[row, col].legend(
                handles=[
                    mpatches.Patch(color="tab:blue", label=label1, alpha=0.5),
                    mpatches.Patch(color="tab:orange", label=label2, alpha=0.5),
                ],
                fontsize=25,
            )

        plt.savefig(
            f"./results/economic_results/half/images/three_sectors/doinfo_three_sectors_{target1}_{target2}_tw_{tw}.pdf",
            dpi=600,
            bbox_inches="tight",
        )

<a name="comparison_lengths"></a>
## Comparison different lengths

<a name="comparison_f_h"></a>
### Comparison full-half

In this section I compare the results obtained considering the entire length with those obtained considering only half of it. In each plot a target is considered; on the y-axis there is the JS-divergence (for full length) between the distributions in which the triplet is all in the target and in which one point is in the target and the other two in another sector. On the x axis there is the similar quantity but for half the length.

In [19]:
df_full = pd.read_csv("./results/economic_results/full/result_full.csv", index_col=0)
df_half = pd.read_csv("./results/economic_results/half/result_full.csv", index_col=0)

In [20]:
df_full

,Industrials,Basic Materials,Financials,Consumer Services,Health Care,Utilities,Oil & Gas,Consumer Goods
Industrials,NaN,0.122308,0.214089,0.052325,0.091460,0.102860,0.084615,0.219743
Basic Materials,0.138274,NaN,0.155638,0.044207,0.146545,0.053517,0.081695,0.184968
Financials,0.297841,0.134393,NaN,0.587818,0.111515,0.014164,0.274718,0.458693
Consumer Services,0.251773,0.058521,0.158145,NaN,0.095770,0.090858,0.057751,0.215983
Health Care,0.272321,0.122780,0.293127,0.034621,NaN,0.805321,0.067858,0.565578
Utilities,0.182089,0.106477,0.047107,0.242159,0.131293,NaN,0.140260,0.091074
Oil & Gas,0.016285,0.179795,0.207017,0.191269,0.205281,0.014110,NaN,0.212679
Consumer Goods,0.223213,0.318661,0.111528,0.435974,0.478561,0.519101,0.416441,NaN


In [21]:
df_half

,Industrials,Basic Materials,Financials,Consumer Services,Health Care,Utilities,Oil & Gas,Consumer Goods
Industrials,NaN,0.269968,0.116018,0.308068,0.371273,0.238282,0.070591,0.311161
Basic Materials,0.096748,NaN,0.144451,0.070495,0.250676,0.190151,0.094814,0.157143
Financials,0.056859,0.097640,NaN,0.037793,0.082591,0.283933,0.089403,0.415097
Consumer Services,0.197348,0.349489,0.143480,NaN,0.139099,0.079083,0.095869,0.136427
Health Care,0.050651,0.101401,0.076487,0.216035,NaN,0.079241,0.025457,0.109977
Utilities,0.133270,0.138424,0.166786,0.327488,0.071730,NaN,0.128444,0.167022
Oil & Gas,0.049689,0.060586,0.137608,0.120026,0.202821,0.087824,NaN,0.129750
Consumer Goods,0.110282,0.096026,0.186354,0.178672,0.660940,0.397908,0.033815,NaN


In [22]:
name_rows = df_full.index.tolist()
name_columns = df_full.columns.tolist()

In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE SCATTER PLOTS

for row in name_rows:
    columns = [column for column in name_columns if column != row]

    x = [df_half.loc[row, column] for column in columns]
    y = [df_full.loc[row, column] for column in columns]
    
    x_values = np.linspace(min(x), max(x), 100)
    plt.plot(x_values, x_values, color="red", linewidth=1.5)

    plt.scatter(x, y)
    plt.xlabel("JS-div half length")
    plt.ylabel("JS-div full length")
    plt.title(f"Plot for target = {row}")
    plt.savefig(
        f"./results/economic_results/comparison/images/JS_div_two_sectors_target_{row}.pdf",
        dpi=600,
        bbox_inches="tight",
    )
    plt.show()

In [25]:
above_bisector = []
below_bisector = []

for row in name_rows:
    above_count = 0
    columns = [column for column in name_columns if column != row]

    x = [df_half.loc[row, column] for column in columns]
    y = [df_full.loc[row, column] for column in columns]
    
    for i in range(len(x)):
        if x[i] < y[i]:
            above_count += 1
            
    above_bisector.append(above_count)
    below_bisector.append(len(x) - above_count)
    

df_comparison = pd.DataFrame(
    [above_bisector, below_bisector], index=["+", "-"], columns=name_columns
)

df_comparison

,Industrials,Basic Materials,Financials,Consumer Services,Health Care,Utilities,Oil & Gas,Consumer Goods
+,2,3,6,4,6,3,5,5
-,5,4,1,3,1,4,2,2


<a name="creation_dataset"></a>
### Creation dataset

I consider the performance of the JS-div as the length of the TS varies.

Full length

In [26]:
# DO NOT RUN IF YOU ALREADY HAVE THE DATA

length = len(binarized_financial_ts)

for target in sectors:
    other_sectors = [sector for sector in sectors if sector != target]

    for i, sector in enumerate(other_sectors):
        doinfo_one_sector = np.load(
            f"./results/economic_results/full/do_info/doinfo_one_sector_{target}_tw_{tw}.npy"
        )
        np.save(
            f"./results/economic_results/comparison/do_info/one_sector_{target}_length_{length}.npy",
            doinfo_one_sector,
        )

        doinfo_two_sectors = np.load(
            f"./results/economic_results/full/do_info/doinfo_two_sectors_{target}_{sector}_tw_{tw}.npy"
        )
        np.save(
            f"./results/economic_results/comparison/do_info/two_sectors_{target}_{sector}_length_{length}.npy",
            doinfo_two_sectors,
        )

Half length

In [27]:
# DO NOT RUN IF YOU ALREADY HAVE THE DATA

length = int(len(binarized_financial_ts) / 2)

for target in sectors:
    other_sectors = [sector for sector in sectors if sector != target]

    for i, sector in enumerate(other_sectors):
        doinfo_one_sector = np.load(
            f"./results/economic_results/half/do_info/doinfo_one_sector_{target}_tw_{tw}.npy"
        )
        np.save(
            f"./results/economic_results/comparison/do_info/one_sector_{target}_length_{length}.npy",
            doinfo_one_sector,
        )

        doinfo_two_sectors = np.load(
            f"./results/economic_results/half/do_info/doinfo_two_sectors_{target}_{sector}_tw_{tw}.npy"
        )
        np.save(
            f"./results/economic_results/comparison/do_info/two_sectors_{target}_{sector}_length_{length}.npy",
            doinfo_two_sectors,
        )

Other lengths

In [28]:
# DO NOT RUN IF YOU ALREADY HAVE THE DATA

lengths = [540, 1080, 1620, 2160, 3240, 3780, 4320, 4860]

for length in lengths:
    for target in sectors:
        other_sectors = [sector for sector in sectors if sector != target]
        for i, sector in enumerate(other_sectors):

            if i == 0:
                doinfo_one_sector = get_do_infos_same(
                    target, binarized_financial_ts[:length]
                )
                np.save(
                    f"./results/economic_results/comparison/do_info/one_sector_{target}_length_{length}.npy",
                    doinfo_one_sector,
                )

            doinfo_two_sectors = get_do_infos_two_different(
                target, sector, binarized_financial_ts[:length]
            )
            np.save(
                f"./results/economic_results/comparison/do_info/two_sectors_{target}_{sector}_length_{length}.npy",
                doinfo_two_sectors,
            )

>>> Industrials
Length target_stocks: 20
Length triplets same sector: 35


  0%|          | 0/35 [00:00<?, ?it/s]

>>> Industrials - Basic Materials
Length another_stocks: 8
Length triplets two sectors: 35


  0%|          | 0/35 [00:00<?, ?it/s]

>>> Industrials - Financials
Length another_stocks: 38
Length triplets two sectors: 35


...

  0%|          | 0/35 [00:00<?, ?it/s]

>>> Consumer Goods - Utilities
Length another_stocks: 7
Length triplets two sectors: 35


  0%|          | 0/35 [00:00<?, ?it/s]

>>> Consumer Goods - Oil & Gas
Length another_stocks: 7
Length triplets two sectors: 35


  0%|          | 0/35 [00:00<?, ?it/s]

<a name="comparison_industrials"></a>
#### Comparison Industrials

In [45]:
df_industrials = pd.DataFrame(
    columns=[
        "Basic Materials",
        "Financials",
        "Consumer Services",
        "Health Care",
        "Utilities",
        "Oil & Gas",
        "Consumer Goods",
    ]
)

In [46]:
lengths = [
    int(len(binarized_financial_ts) * i)
    for i in np.linspace(0.1, 1, 10)
]

In [47]:
# DO NOT RUN IF YOU ALREADY HAVE THE DATAFRAME

for length in lengths:
    for target in ["Industrials"]:
        other_sectors = [sector for sector in sectors if sector != target]

        for i, sector in enumerate(other_sectors):
            doinfo_one_sector = np.load(
                f"./results/economic_results/comparison/do_info/one_sector_{target}_length_{length}.npy"
            )
            doinfo_two_sectors = np.load(
                f"./results/economic_results/comparison/do_info/two_sectors_{target}_{sector}_length_{length}.npy"
            )

            df_industrials.at[length, sector] = JS_div(doinfo_one_sector, doinfo_two_sectors)
            
df_industrials.to_csv("./results/economic_results/comparison/length_Industrials.csv", index=True)

In [48]:
df_industrials = pd.read_csv("./results/economic_results/comparison/length_Industrials.csv", index_col=0)
df_industrials

,Basic Materials,Financials,Consumer Services,Health Care,Utilities,Oil & Gas,Consumer Goods
540,0.237938,0.290896,0.168913,0.221155,0.129496,0.289258,0.060225
1080,0.095673,0.026834,0.125723,0.011031,0.104654,0.040729,0.055406
1620,0.107160,0.141756,0.233492,0.254492,0.636388,0.056553,0.156158
2160,0.294695,0.197886,0.122706,0.385376,0.205085,0.230528,0.417641
2700,0.269968,0.116018,0.308068,0.371273,0.238282,0.070591,0.311161
3240,0.144985,0.199808,0.132998,0.212263,0.058871,0.227055,0.210336
3780,0.208662,0.041249,0.088355,0.055319,0.148649,0.129300,0.344918
4320,0.133865,0.080710,0.167774,0.094971,0.056954,0.063473,0.117314
4860,0.071358,0.187431,0.291538,0.016148,0.084509,0.142853,0.196774
5400,0.122308,0.214089,0.052325,0.091460,0.102860,0.084615,0.219743


In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE PLOT

num_rows = len(df_industrials.columns)
num_cols = 1

fig, axs = plt.subplots(num_rows, num_cols, figsize=(10, 25))

for i, column in enumerate(df_industrials.columns):
    label = str(f"{column}")
    axs[i].scatter(df_industrials.index, df_industrials[column], label=label)

    axs[i].legend(fontsize=13)
    axs[i].set_ylabel("JS-div", fontsize=14)

    if i == num_rows - 1:
        axs[i].set_xlabel("Length", fontsize=14)

fig.suptitle("Industrials", fontsize=22)
plt.subplots_adjust(top=0.96)

plt.savefig(
    f"./results/economic_results/comparison/images/JS_div_different_lengths_Industrials.pdf",
    dpi=600,
    bbox_inches="tight",
)

plt.show()

<a name="comparison_basic_materials"></a>
#### Comparison Basic Materials

In [50]:
df__basic_materials = pd.DataFrame(
    columns=[
        "Industrials",
        "Financials",
        "Consumer Services",
        "Health Care",
        "Utilities",
        "Oil & Gas",
        "Consumer Goods",
    ]
)

In [51]:
# DO NOT RUN IF YOU ALREADY HAVE THE DATAFRAME

for length in lengths:
    for target in ["Basic Materials"]:
        other_sectors = [sector for sector in sectors if sector != target]

        for i, sector in enumerate(other_sectors):
            doinfo_one_sector = np.load(
                f"./results/economic_results/comparison/do_info/one_sector_{target}_length_{length}.npy"
            )
            doinfo_two_sectors = np.load(
                f"./results/economic_results/comparison/do_info/two_sectors_{target}_{sector}_length_{length}.npy"
            )

            df__basic_materials.at[length, sector] = JS_div(doinfo_one_sector, doinfo_two_sectors)
            
df__basic_materials.to_csv("./results/economic_results/comparison/length_Basic_Materials.csv", index=True)

In [52]:
df__basic_materials = pd.read_csv("./results/economic_results/comparison/length_Basic_Materials.csv", index_col=0)
df__basic_materials

,Industrials,Financials,Consumer Services,Health Care,Utilities,Oil & Gas,Consumer Goods
540,0.081032,0.380973,0.075664,0.335837,0.334225,0.171204,0.021553
1080,0.138825,0.451077,0.357059,0.175163,0.123425,0.160751,0.308767
1620,0.191901,0.714645,0.269628,0.450079,0.077709,0.230037,0.206995
2160,0.084775,0.259075,0.206611,0.214672,0.135764,0.181673,0.126781
2700,0.096748,0.144451,0.070495,0.250676,0.190151,0.094814,0.157143
3240,0.057478,0.056536,0.054811,0.060867,0.092053,0.063125,0.060339
3780,0.106322,0.071975,0.121036,0.106352,0.119284,0.107669,0.183105
4320,0.081222,0.033977,0.035049,0.089541,0.137927,0.039365,0.168125
4860,0.067517,0.050158,0.011147,0.129689,0.129594,0.101590,0.201987
5400,0.138274,0.155638,0.044207,0.146545,0.053517,0.081695,0.184968


In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE PLOT

num_rows = len(df__basic_materials.columns)
num_cols = 1

fig, axs = plt.subplots(num_rows, num_cols, figsize=(10, 25))

for i, column in enumerate(df__basic_materials.columns):
    label = str(f"{column}")
    axs[i].scatter(df__basic_materials.index, df__basic_materials[column], label=label)

    axs[i].legend(fontsize=13)
    axs[i].set_ylabel("JS-div", fontsize=14)

    if i == num_rows - 1:
        axs[i].set_xlabel("Length", fontsize=14)

fig.suptitle("Basic Materials", fontsize=22)
plt.subplots_adjust(top=0.96)

plt.savefig(
    f"./results/economic_results/comparison/images/JS_div_different_lengths_Basic_Materials.pdf",
    dpi=600,
    bbox_inches="tight",
)

plt.show()

<a name="comparison_financials"></a>
#### Comparison Financials

In [54]:
df_financials = pd.DataFrame(
    columns=[
        "Industrials",
        "Basic Materials",
        "Consumer Services",
        "Health Care",
        "Utilities",
        "Oil & Gas",
        "Consumer Goods",
    ]
)

In [55]:
# DO NOT RUN IF YOU ALREADY HAVE THE DATAFRAME

for length in lengths:
    for target in ["Financials"]:
        other_sectors = [sector for sector in sectors if sector != target]

        for i, sector in enumerate(other_sectors):
            doinfo_one_sector = np.load(
                f"./results/economic_results/comparison/do_info/one_sector_{target}_length_{length}.npy"
            )
            doinfo_two_sectors = np.load(
                f"./results/economic_results/comparison/do_info/two_sectors_{target}_{sector}_length_{length}.npy"
            )

            df_financials.at[length, sector] = JS_div(doinfo_one_sector, doinfo_two_sectors)

df_financials.to_csv("./results/economic_results/comparison/length_Financial.csv", index=True)

In [56]:
df_financials = pd.read_csv("./results/economic_results/comparison/length_Financial.csv", index_col=0)
df_financials

,Industrials,Basic Materials,Consumer Services,Health Care,Utilities,Oil & Gas,Consumer Goods
540,0.111597,0.025476,0.257380,0.035097,0.053989,0.107796,0.014552
1080,0.015624,0.116758,0.178022,0.404487,0.330720,0.078032,0.317233
1620,0.176130,0.062804,0.171156,0.157000,0.047999,0.043958,0.048653
2160,0.267727,0.360247,0.112119,0.090785,0.016171,0.198487,0.154970
2700,0.056859,0.097640,0.037793,0.082591,0.283933,0.089403,0.415097
3240,0.154243,0.148640,0.215709,0.158072,0.255872,0.878518,0.542141
3780,0.083985,0.133829,0.322500,0.047333,0.088753,0.146895,0.090804
4320,0.210367,0.085035,0.048304,0.704780,0.073263,0.205932,0.124943
4860,0.117822,0.163170,0.298634,0.242723,0.136402,0.228321,0.190445
5400,0.297841,0.134393,0.587818,0.111515,0.014164,0.274718,0.458693


In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE PLOT

num_rows = len(df_financials.columns)
num_cols = 1

fig, axs = plt.subplots(num_rows, num_cols, figsize=(10, 25))

for i, column in enumerate(df_financials.columns):
    label = str(f"{column}")
    axs[i].scatter(df_financials.index, df_financials[column], label=label)

    axs[i].legend(fontsize=13)
    axs[i].set_ylabel("JS-div", fontsize=14)

    if i == num_rows - 1:
        axs[i].set_xlabel("Length", fontsize=14)

fig.suptitle("Financials", fontsize=22)
plt.subplots_adjust(top=0.96)

plt.savefig(
    f"./results/economic_results/comparison/images/JS_div_different_lengths_Financials.pdf",
    dpi=600,
    bbox_inches="tight",
)

plt.show()

<a name="comparison_consumer_services"></a>
#### Comparison Consumer Services

In [58]:
df_consumer_services = pd.DataFrame(
    columns=[
        "Industrials",
        "Basic Materials",
        "Financials",
        "Health Care",
        "Utilities",
        "Oil & Gas",
        "Consumer Goods",
    ]
)

In [59]:
# DO NOT RUN IF YOU ALREADY HAVE THE DATAFRAME

for length in lengths:
    for target in ["Consumer Services"]:
        other_sectors = [sector for sector in sectors if sector != target]

        for i, sector in enumerate(other_sectors):
            doinfo_one_sector = np.load(
                f"./results/economic_results/comparison/do_info/one_sector_{target}_length_{length}.npy"
            )
            doinfo_two_sectors = np.load(
                f"./results/economic_results/comparison/do_info/two_sectors_{target}_{sector}_length_{length}.npy"
            )

            df_consumer_services.at[length, sector] = JS_div(doinfo_one_sector, doinfo_two_sectors)
            
df_consumer_services.to_csv("./results/economic_results/comparison/length_Consumer_Services.csv", index=True)

In [60]:
df_consumer_services = pd.read_csv("./results/economic_results/comparison/length_Consumer_Services.csv", index_col=0)
df_consumer_services

,Industrials,Basic Materials,Financials,Health Care,Utilities,Oil & Gas,Consumer Goods
540,1.246342,0.179371,0.316239,0.184459,0.588726,0.112430,0.162135
1080,0.096979,0.613024,0.162935,0.060288,0.021314,0.057559,0.305761
1620,0.055757,0.082892,0.112827,0.224105,0.112743,0.064794,0.113191
2160,0.160509,0.036899,0.137742,0.134081,0.070987,0.058460,0.161789
2700,0.197348,0.349489,0.143480,0.139099,0.079083,0.095869,0.136427
3240,0.492346,0.875921,0.444437,0.136206,0.062121,0.032191,0.332223
3780,0.057287,0.085243,0.154287,0.113722,0.169207,0.145915,0.125489
4320,0.230512,0.146738,0.166253,0.710403,0.037580,0.132394,0.154590
4860,0.045215,0.117906,0.118461,0.055461,0.191958,0.163717,0.084087
5400,0.251773,0.058521,0.158145,0.095770,0.090858,0.057751,0.215983


In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE PLOT

num_rows = len(df_consumer_services.columns)
num_cols = 1

fig, axs = plt.subplots(num_rows, num_cols, figsize=(10, 25))

for i, column in enumerate(df_consumer_services.columns):
    label = str(f"{column}")
    axs[i].scatter(df_consumer_services.index, df_consumer_services[column], label=label)

    axs[i].legend(fontsize=13)
    axs[i].set_ylabel("JS-div", fontsize=14)

    if i == num_rows - 1:
        axs[i].set_xlabel("Length", fontsize=14)

fig.suptitle("Consumer Services", fontsize=22)
plt.subplots_adjust(top=0.96)

plt.savefig(
    f"./results/economic_results/comparison/images/JS_div_different_lengths_Consumer_Services.pdf",
    dpi=600,
    bbox_inches="tight",
)

plt.show()

<a name="comparison_health_care"></a>
#### Comparison Health Care

In [62]:
df_health_care = pd.DataFrame(
    columns=[
        "Industrials",
        "Basic Materials",
        "Financials",
        "Consumer Services",
        "Utilities",
        "Oil & Gas",
        "Consumer Goods",
    ]
)

In [63]:
# DO NOT RUN IF YOU ALREADY HAVE THE DATAFRAME

for length in lengths:
    for target in ["Health Care"]:
        other_sectors = [sector for sector in sectors if sector != target]

        for i, sector in enumerate(other_sectors):
            doinfo_one_sector = np.load(
                f"./results/economic_results/comparison/do_info/one_sector_{target}_length_{length}.npy"
            )
            doinfo_two_sectors = np.load(
                f"./results/economic_results/comparison/do_info/two_sectors_{target}_{sector}_length_{length}.npy"
            )

            df_health_care.at[length, sector] = JS_div(doinfo_one_sector, doinfo_two_sectors)
            
df_health_care.to_csv("./results/economic_results/comparison/length_Health_Care.csv", index=True)

In [64]:
df_health_care = pd.read_csv("./results/economic_results/comparison/length_Health_Care.csv", index_col=0)
df_health_care

,Industrials,Basic Materials,Financials,Consumer Services,Utilities,Oil & Gas,Consumer Goods
540,0.101304,0.284673,0.104652,0.088483,0.150381,0.194953,0.135158
1080,0.418981,0.437179,0.016498,0.175799,0.118674,0.132400,0.276593
1620,0.218921,0.361210,0.308763,0.529758,0.047251,0.325215,0.318497
2160,0.158084,0.115810,0.382182,0.091618,0.100715,0.037131,0.061156
2700,0.050651,0.101401,0.076487,0.216035,0.079241,0.025457,0.109977
3240,0.191300,0.056427,0.165446,0.221082,0.450306,0.032946,0.139740
3780,0.667763,0.026498,0.247510,0.068005,0.092030,0.199079,0.375863
4320,0.069693,0.125931,0.040633,0.296324,0.216360,0.143094,0.271087
4860,0.064865,0.172484,0.085089,0.305929,0.251835,0.421121,0.006832
5400,0.272321,0.122780,0.293127,0.034621,0.805321,0.067858,0.565578


In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE PLOT

num_rows = len(df_health_care.columns)
num_cols = 1

fig, axs = plt.subplots(num_rows, num_cols, figsize=(10, 25))

for i, column in enumerate(df_health_care.columns):
    label = str(f"{column}")
    axs[i].scatter(df_health_care.index, df_health_care[column], label=label)

    axs[i].legend(fontsize=13)
    axs[i].set_ylabel("JS-div", fontsize=14)

    if i == num_rows - 1:
        axs[i].set_xlabel("Length", fontsize=14)

fig.suptitle("Health Care", fontsize=22)
plt.subplots_adjust(top=0.96)

plt.savefig(
    f"./results/economic_results/comparison/images/JS_div_different_lengths_Health_Care.pdf",
    dpi=600,
    bbox_inches="tight",
)

plt.show()

<a name="comparison_utilities"></a>
#### Comparison Utilities

In [66]:
df_utilities = pd.DataFrame(
    columns=[
        "Industrials",
        "Basic Materials",
        "Financials",
        "Consumer Services",
        "Health Care",
        "Oil & Gas",
        "Consumer Goods",
    ]
)

In [67]:
# DO NOT RUN IF YOU ALREADY HAVE THE DATAFRAME

for length in lengths:
    for target in ["Utilities"]:
        other_sectors = [sector for sector in sectors if sector != target]

        for i, sector in enumerate(other_sectors):
            doinfo_one_sector = np.load(
                f"./results/economic_results/comparison/do_info/one_sector_{target}_length_{length}.npy"
            )
            doinfo_two_sectors = np.load(
                f"./results/economic_results/comparison/do_info/two_sectors_{target}_{sector}_length_{length}.npy"
            )

            df_utilities.at[length, sector] = JS_div(doinfo_one_sector, doinfo_two_sectors)
            
df_utilities.to_csv("./results/economic_results/comparison/length_Utilities.csv", index=True)

In [68]:
df_utilities = pd.read_csv("./results/economic_results/comparison/length_Utilities.csv", index_col=0)
df_utilities

,Industrials,Basic Materials,Financials,Consumer Services,Health Care,Oil & Gas,Consumer Goods
540,0.122809,0.175325,0.043810,0.069035,0.244016,0.257514,0.176680
1080,0.138993,0.353341,0.196876,0.122209,0.041759,0.174350,0.166150
1620,0.251628,0.263643,0.732662,0.142729,0.262115,0.203304,0.269672
2160,0.091617,0.158802,0.506709,0.675205,0.204253,0.223722,0.391948
2700,0.133270,0.138424,0.166786,0.327488,0.071730,0.128444,0.167022
3240,0.099565,0.114888,0.250475,0.449826,0.216596,0.135650,0.134679
3780,0.081339,0.310872,0.070804,0.216966,0.322694,0.074584,0.130843
4320,0.080093,0.242689,0.132279,0.023848,0.097610,0.066710,0.086938
4860,0.068947,0.292181,0.047773,0.560268,0.231073,0.037077,0.031570
5400,0.182089,0.106477,0.047107,0.242159,0.131293,0.140260,0.091074


In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE PLOT

num_rows = len(df_utilities.columns)
num_cols = 1

fig, axs = plt.subplots(num_rows, num_cols, figsize=(10, 25))

for i, column in enumerate(df_utilities.columns):
    label = str(f"{column}")
    axs[i].scatter(df_utilities.index, df_utilities[column], label=label)

    axs[i].legend(fontsize=13)
    axs[i].set_ylabel("JS-div", fontsize=14)

    if i == num_rows - 1:
        axs[i].set_xlabel("Length", fontsize=14)

fig.suptitle("Utilities", fontsize=22)
plt.subplots_adjust(top=0.96)

plt.savefig(
    f"./results/economic_results/comparison/images/JS_div_different_lengths_Utilities.pdf",
    dpi=600,
    bbox_inches="tight",
)

plt.show()

<a name="comparison_oil_gas"></a>
#### Comparison Oil & Gas

In [70]:
df_oil_gas = pd.DataFrame(
    columns=[
        "Industrials",
        "Basic Materials",
        "Financials",
        "Consumer Services",
        "Health Care",
        "Utilities",
        "Consumer Goods",
    ]
)

In [71]:
# DO NOT RUN IF YOU ALREADY HAVE THE DATAFRAME

for length in lengths:
    for target in ["Oil & Gas"]:
        other_sectors = [sector for sector in sectors if sector != target]

        for i, sector in enumerate(other_sectors):
            doinfo_one_sector = np.load(
                f"./results/economic_results/comparison/do_info/one_sector_{target}_length_{length}.npy"
            )
            doinfo_two_sectors = np.load(
                f"./results/economic_results/comparison/do_info/two_sectors_{target}_{sector}_length_{length}.npy"
            )

            df_oil_gas.at[length, sector] = JS_div(doinfo_one_sector, doinfo_two_sectors)
            
df_oil_gas.to_csv("./results/economic_results/comparison/length_Oil_Gas.csv", index=True)

In [72]:
df_oil_gas = pd.read_csv("./results/economic_results/comparison/length_Oil_Gas.csv", index_col=0)
df_oil_gas

,Industrials,Basic Materials,Financials,Consumer Services,Health Care,Utilities,Consumer Goods
540,0.394316,0.046468,0.132091,0.211189,0.127168,0.064964,0.107234
1080,0.368040,0.334635,0.635351,0.277308,0.562990,0.382542,0.239845
1620,0.507564,0.202385,0.491652,0.392303,0.563466,0.173760,0.191688
2160,0.210871,0.106405,0.345632,0.111391,0.429586,0.175299,0.509746
2700,0.049689,0.060586,0.137608,0.120026,0.202821,0.087824,0.129750
3240,0.158467,0.127956,0.109397,0.108542,0.219105,0.021544,0.173290
3780,0.119653,0.060483,0.252743,0.225094,0.213778,0.032207,0.266900
4320,0.061238,0.069340,0.607399,0.038695,0.034413,0.034973,0.038712
4860,0.031507,0.073589,0.138007,0.107512,0.236796,0.211319,0.265699
5400,0.016285,0.179795,0.207017,0.191269,0.205281,0.014110,0.212679


In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE PLOT

num_rows = len(df_oil_gas.columns)
num_cols = 1

fig, axs = plt.subplots(num_rows, num_cols, figsize=(10, 25))

for i, column in enumerate(df_oil_gas.columns):
    label = str(f"{column}")
    axs[i].scatter(df_oil_gas.index, df_oil_gas[column], label=label)

    axs[i].legend(fontsize=13)
    axs[i].set_ylabel("JS-div", fontsize=14)

    if i == num_rows - 1:
        axs[i].set_xlabel("Length", fontsize=14)

fig.suptitle("Oil & Gas", fontsize=22)
plt.subplots_adjust(top=0.96)

plt.savefig(
    f"./results/economic_results/comparison/images/JS_div_different_lengths_Oil_Gas.pdf",
    dpi=600,
    bbox_inches="tight",
)

plt.show()

<a name="comparison_consumer_goods"></a>
#### Comparison Consumer Goods

In [74]:
df_consumer_goods = pd.DataFrame(
    columns=[
        "Industrials",
        "Basic Materials",
        "Financials",
        "Consumer Services",
        "Health Care",
        "Utilities",
        "Oil & Gas",
    ]
)

In [75]:
# DO NOT RUN IF YOU ALREADY HAVE THE DATAFRAME

for length in lengths:
    for target in ["Consumer Goods"]:
        other_sectors = [sector for sector in sectors if sector != target]

        for i, sector in enumerate(other_sectors):
            doinfo_one_sector = np.load(
                f"./results/economic_results/comparison/do_info/one_sector_{target}_length_{length}.npy"
            )
            doinfo_two_sectors = np.load(
                f"./results/economic_results/comparison/do_info/two_sectors_{target}_{sector}_length_{length}.npy"
            )

            df_consumer_goods.at[length, sector] = JS_div(doinfo_one_sector, doinfo_two_sectors)
            
df_consumer_goods.to_csv("./results/economic_results/comparison/length_Consumer_Goods.csv", index=True)

In [76]:
df_consumer_goods = pd.read_csv("./results/economic_results/comparison/length_Consumer_Goods.csv", index_col=0)
df_consumer_goods

,Industrials,Basic Materials,Financials,Consumer Services,Health Care,Utilities,Oil & Gas
540,0.207201,0.012062,0.025494,0.260132,0.219128,0.212006,0.139537
1080,0.044501,0.150003,0.058690,0.144147,0.098283,0.020416,0.451871
1620,0.151708,0.182730,0.024641,0.229652,0.187758,0.130214,0.086772
2160,0.110883,0.323160,0.760029,0.375934,0.555935,0.176890,0.258789
2700,0.110282,0.096026,0.186354,0.178672,0.660940,0.397908,0.033815
3240,0.173235,0.070333,1.022119,0.202840,0.343052,0.199805,0.272402
3780,0.375807,0.078080,0.216999,0.213575,0.233843,0.161653,0.385926
4320,0.337101,0.168514,0.090697,0.217418,0.210076,0.308004,0.238862
4860,0.084147,0.135407,0.150417,0.476784,0.817605,0.230180,0.149949
5400,0.223213,0.318661,0.111528,0.435974,0.478561,0.519101,0.416441


In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE PLOT

num_rows = len(df_consumer_goods.columns)
num_cols = 1

fig, axs = plt.subplots(num_rows, num_cols, figsize=(10, 25))

for i, column in enumerate(df_consumer_goods.columns):
    label = str(f"{column}")
    axs[i].scatter(df_consumer_goods.index, df_consumer_goods[column], label=label)

    axs[i].legend(fontsize=13)
    axs[i].set_ylabel("JS-div", fontsize=14)

    if i == num_rows - 1:
        axs[i].set_xlabel("Length", fontsize=14)

fig.suptitle("Consumer Goods", fontsize=22)
plt.subplots_adjust(top=0.96)

plt.savefig(
    f"./results/economic_results/comparison/images/JS_div_different_lengths_Consumer_Goods.pdf",
    dpi=600,
    bbox_inches="tight",
)

plt.show()